In [1]:
import math
import h5py
import numpy as np 
import netCDF4 as nc
from netCDF4 import Dataset
from sklearn.linear_model import LinearRegression
import xarray as xr
import matplotlib.pyplot as plt
import time
import pandas as pd
import datetime
import random

import os
import torch
print(torch.__version__)
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch import from_numpy
from torch import manual_seed as torch_manual_seed
from torch.cuda import max_memory_allocated, set_device, manual_seed_all
from torch.backends import cudnn

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from functools import partial

2.2.0+cu118


# TODO

Run model on test data 

Get R2 for test data per region on trained model

Build indicative plots for 1 simulation (preds and actual)



# CNN

### Prepare Data for CNN in PyTorch

In [9]:
ds2 = xr.open_dataset("Detrended_SST.nc", decode_times=False)
ds2

<xarray.Dataset>
Dimensions:  (lat: 192, lon: 288, time: 7400)
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) int64 0 366 731 1096 1461 ... 25203 25568 25933 26298 26664
Data variables:
    SST      (time, lat, lon) float32 ...

In [22]:
PRECT = pd.read_csv('Detrended_PRECT.csv')
PRECT = PRECT.set_index(['simulation', 'year', 'region'])
PRECT.head()

PRECT
simulation year region            
1001.001   1940 Central   1.696816
                North    -4.384861
                South     2.359652
           1941 Central   7.414276
                North    15.164542

In [23]:
y = PRECT.to_numpy()
y.shape

(22200, 1)

In [24]:
y

array([[ 1.69681626],
       [-4.38486059],
       [ 2.35965193],
       ...,
       [ 1.42676119],
       [-2.24789536],
       [ 2.00405384]])

In [25]:
y = y.reshape(-1, 3)
y

array([[ 1.69681626, -4.38486059,  2.35965193],
       [ 7.414276  , 15.16454221, -0.27659118],
       [ 1.00511413, -0.24650046,  1.69304477],
       ...,
       [-2.79026449,  2.59703625, -4.35883811],
       [-6.19666334, -2.17127014, -4.9022737 ],
       [ 1.42676119, -2.24789536,  2.00405384]])

In [26]:
y.shape

(7400, 3)

In [27]:
X = ds2['SST'].to_numpy()
X.shape

(7400, 192, 288)

In [28]:
def setup_seed(seed):
    torch_manual_seed(seed)
    manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    cudnn.deterministic = True

SEED = 42
setup_seed(SEED)

In [29]:
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.1, random_state=49)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size= 0.1, random_state=49)

#Create Dataloader Objects for PyTorch
#BATCH_SIZE = 16
BATCH_SIZE = 32

X_train = from_numpy(X_train).float()
y_train = from_numpy(y_train).float()

X_val = from_numpy(X_val).float()
y_val = from_numpy(y_val).float()

X_test = from_numpy(X_test).float()
y_test = from_numpy(y_test).float()

dataset_train = TensorDataset(X_train, y_train)
dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)

dataset_valid = TensorDataset(X_val, y_val)
dataloader_valid = DataLoader(dataset_valid, batch_size=BATCH_SIZE, shuffle=False)

dataset_test = TensorDataset(X_test, y_test)
dataloader_test = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False)

#Check shape of resulting objects
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)


torch.Size([5994, 192, 288])
torch.Size([740, 192, 288])
torch.Size([666, 192, 288])
torch.Size([5994, 3])
torch.Size([740, 3])
torch.Size([666, 3])


In [ ]:
for batch in dataloader_train:
    predictor_batch, target_batch = batch
    print(predictor_batch.shape)
    break

torch.Size([16, 192, 288])


### Build CNN

In [30]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, stride = (2,2), kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 32, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(32, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 6 * 9, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 3)  # 3 output values

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = self.pool(torch.relu(self.conv4(x)))
        x = x.view(-1, 16 * 6 * 9)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model=Net()
model

Net(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=864, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=3, bias=True)
)

In [31]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
pytorch_total_params

179827

In [32]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
epochs = 5

### Train CNN

In [122]:
%%time 

for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader_train, 0):
        
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.unsqueeze(1)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 16 == 15:    # print every 16 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 16:.3f}')
            running_loss = 0.0

print('Finished Training')

torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
[1,    16] loss: 24.531
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
[1,    32] loss: 26.325
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.Size([16, 16, 6, 9])
torch.S

KeyboardInterrupt: 

### Tune CNN

### Test CNN

## Potential Resources

https://www.nature.com/articles/s41598-020-57897-9

https://www.nature.com/articles/s41586-019-1559-7 
- first architecture option